In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#

In [2]:
df = pd.read_csv('/content/blogs.csv')

In [3]:
df

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [4]:
print(df.head())
print(df.info())
print(df.describe())


                                                Data       Labels
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....  alt.atheism
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...  alt.atheism
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...  alt.atheism
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
None
                                                     Data       Labels
count                                                2000         2000
unique                                               2000           20
top     Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  alt.atheism
freq                    

# DATA Cleaning

In [8]:
import re
import string

def clean_text(Data):
    Data = Data.lower()
    Data = re.sub('\[.*?\]', '', Data)
    Data = re.sub('[%s]' % re.escape(string.punctuation), '', Data)
    Data = re.sub('\w*\d\w*', '', Data)
    Data = re.sub('[‘’“”…]', '', Data)
    Data=re.sub("[0-9""]+","",Data)
    Data = re.sub('\n', '', Data)
    return Data
clean = lambda x: clean_text(x)

In [9]:
df['Data']=df.Data.apply(clean)
df.Data

,Data
0,path from frank odwyernewsgroups altatheismsu...
1,newsgroups altatheismpath from bilokcforumosrh...
2,path cantaloupesrvcscmuedudasnewsharvardedunoc...
3,path from jbrownbatmanbmdtrwcomnewsgroups alta...
4,xref cantaloupesrvcscmuedu newsgroups altath...
...,...
1995,xref cantaloupesrvcscmuedu path from frank ...
1996,xref cantaloupesrvcscmuedu newsgroups talkre...
1997,xref cantaloupesrvcscmuedu path cantaloupesrv...
1998,xref cantaloupesrvcscmuedu path cantaloupesrv...


In [10]:
#Word frequency
freq = pd.Series(' '.join(df['Data']).split()).value_counts()[:20]
freq

,count
the,24321
to,12485
of,11863
a,10459
and,9891
in,7964
is,7442
i,6992
that,6448
it,4906


In [11]:
#removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Data'] = df['Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Data,Labels
0,path frank odwyernewsgroups altatheismsubject ...,alt.atheism
1,newsgroups altatheismpath bilokcforumosrheedu ...,alt.atheism
2,path cantaloupesrvcscmuedudasnewsharvardedunoc...,alt.atheism
3,path jbrownbatmanbmdtrwcomnewsgroups altatheis...,alt.atheism
4,xref cantaloupesrvcscmuedu newsgroups altathei...,alt.atheism
...,...,...
1995,xref cantaloupesrvcscmuedu path frank odwyerne...,talk.religion.misc
1996,xref cantaloupesrvcscmuedu newsgroups talkreli...,talk.religion.misc
1997,xref cantaloupesrvcscmuedu path cantaloupesrvc...,talk.religion.misc
1998,xref cantaloupesrvcscmuedu path cantaloupesrvc...,talk.religion.misc


In [12]:
df.shape

(2000, 2)

In [13]:
#word frequency after removal of stopwords
freq = pd.Series(' '.join(df['Data']).split()).value_counts()[:20]
freq

,count
apr,1939
would,1443
one,1432
article,1276
date,1239
newsgroups,1148
dont,1085
path,1065
people,978
like,976


In [14]:
#count vectoriser tells the frequency of word
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vectorizer = CountVectorizer(min_df=1,max_df=0.9)
X = vectorizer.fit_transform(df['Data'])
word_freq_df = pd.DataFrame({'term': vectorizer.get_feature_names_out(), 'occurrences':np.asarray(X.sum(axis=0)).ravel().tolist()})
word_freq_df['frequency'] = word_freq_df['occurrences']/np.sum(word_freq_df['occurrences'])

In [15]:
word_freq_df.head(30)

,term,occurrences,frequency
0,aa,8,0.000026
1,aaa,3,0.000010
2,aaaaarrrrghearlier,1,0.000003
3,aaand,1,0.000003
4,aaboston,1,0.000003
5,aalan,1,0.000003
6,aalternate,1,0.000003
7,aaltonen,1,0.000003
8,aaltonenorganization,1,0.000003
9,aaltonensubject,1,0.000003


In [16]:
#TFLDF -term frequency inverse document frequencyt
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True)
doc_vec = vectorizer.fit_transform(df['Data'])
names_features = vectorizer.get_feature_names_out()
dense = doc_vec.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=names_features)


In [17]:
df

,Data,Labels
0,path frank odwyernewsgroups altatheismsubject ...,alt.atheism
1,newsgroups altatheismpath bilokcforumosrheedu ...,alt.atheism
2,path cantaloupesrvcscmuedudasnewsharvardedunoc...,alt.atheism
3,path jbrownbatmanbmdtrwcomnewsgroups altatheis...,alt.atheism
4,xref cantaloupesrvcscmuedu newsgroups altathei...,alt.atheism
...,...,...
1995,xref cantaloupesrvcscmuedu path frank odwyerne...,talk.religion.misc
1996,xref cantaloupesrvcscmuedu newsgroups talkreli...,talk.religion.misc
1997,xref cantaloupesrvcscmuedu path cantaloupesrvc...,talk.religion.misc
1998,xref cantaloupesrvcscmuedu path cantaloupesrvc...,talk.religion.misc


In [18]:
df.head()

,Data,Labels
0,path frank odwyernewsgroups altatheismsubject ...,alt.atheism
1,newsgroups altatheismpath bilokcforumosrheedu ...,alt.atheism
2,path cantaloupesrvcscmuedudasnewsharvardedunoc...,alt.atheism
3,path jbrownbatmanbmdtrwcomnewsgroups altatheis...,alt.atheism
4,xref cantaloupesrvcscmuedu newsgroups altathei...,alt.atheism


In [19]:
def split_into_words(i):
    return [word for word in i.split(" ")]

In [20]:
from sklearn.model_selection import train_test_split

email_train,email_test = train_test_split(df,test_size=0.3)

In [21]:
email_train

,Data,Labels
151,xref cantaloupesrvcscmuedu newsgroups path can...,comp.graphics
1133,xref cantaloupesrvcscmuedu newsgroups scicrypt...,sci.crypt
1044,path organization central michigan universityd...,rec.sport.hockey
130,newsgroups compgraphicspath cantaloupesrvcscmu...,comp.graphics
832,newsgroups recmotorcyclespath bdfluentdartmout...,rec.motorcycles
...,...,...
633,path cantaloupesrvcscmuedurochestercornellbatc...,misc.forsale
325,path ken nakata cis stntnewsgroups compsysibmp...,comp.sys.ibm.pc.hardware
735,newsgroups recautospath jkjecwestminsteracuk s...,rec.autos
1325,path gebcspittedu gordon banksnewsgroups scime...,sci.med


In [22]:
emails_bow = CountVectorizer(analyzer=split_into_words).fit(df.Data)

In [23]:
emails_bow

CountVectorizer(analyzer=<function split_into_words at 0x7a72720ab760>)

In [24]:
all_emails_matrix = emails_bow.transform(df.Data)
all_emails_matrix.shape

(2000, 62833)

In [25]:
train_emails_matrix = emails_bow.transform(email_train.Data)
train_emails_matrix.shape # (3891,8175)

# For testing messages
test_emails_matrix = emails_bow.transform(email_test.Data)
test_emails_matrix.shape # (1668,8175)

(600, 62833)

In [26]:
from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_emails_matrix,email_train.Labels)
train_pred_m = classifier_mb.predict(train_emails_matrix)
accuracy_train_m = np.mean(train_pred_m==email_train.Labels)
print(accuracy_train_m) # 98%

test_pred_m = classifier_mb.predict(test_emails_matrix)
accuracy_test_m = np.mean(test_pred_m==email_test.Labels) # 96%
print(accuracy_test_m)


0.9957142857142857
0.6733333333333333


#Using TFIDF

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

# Learning Term weighting and normalizing on entire emails
tfidf_transformer = TfidfTransformer().fit(all_emails_matrix)

# Preparing TFIDF for train emails
train_tfidf = tfidf_transformer.transform(train_emails_matrix)

train_tfidf.shape # (3891, 6661)

# Preparing TFIDF for test emails
test_tfidf = tfidf_transformer.transform(test_emails_matrix)

test_tfidf.shape #  (1668, 6661)



(600, 62833)

In [28]:
 #Preparing a naive bayes model on training data set

from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_tfidf,email_train.Labels)
train_pred_m = classifier_mb.predict(train_tfidf)
accuracy_train_m = np.mean(train_pred_m==email_train.Labels) # 96%
print(accuracy_train_m)
test_pred_m = classifier_mb.predict(test_tfidf)
accuracy_test_m = np.mean(test_pred_m==email_test.Labels) # 96%
print(accuracy_test_m)



0.9871428571428571
0.73


In [29]:
classifier_gb = GB()
classifier_gb.fit(train_tfidf.toarray(),email_train.Labels.values) # we need to convert tfidf into array format which is compatible for gaussian naive bayes
train_pred_g = classifier_gb.predict(train_tfidf.toarray())
accuracy_train_g = np.mean(train_pred_g==email_train.Labels) # 95%
test_pred_g = classifier_gb.predict(test_tfidf.toarray())
accuracy_test_g = np.mean(test_pred_g==email_test.Labels) # 88%


In [30]:
accuracy_test_g

0.6583333333333333

In [31]:
accuracy_train_g

1.0

In [32]:
from sklearn.metrics import classification_report
print(classification_report(train_pred_m,email_train.Labels))

                          precision    recall  f1-score   support

             alt.atheism       0.99      0.88      0.93        86
           comp.graphics       1.00      1.00      1.00        71
 comp.os.ms-windows.misc       1.00      1.00      1.00        69
comp.sys.ibm.pc.hardware       1.00      1.00      1.00        65
   comp.sys.mac.hardware       1.00      1.00      1.00        69
          comp.windows.x       1.00      0.99      0.99        72
            misc.forsale       1.00      1.00      1.00        63
               rec.autos       1.00      1.00      1.00        70
         rec.motorcycles       1.00      1.00      1.00        74
      rec.sport.baseball       1.00      1.00      1.00        73
        rec.sport.hockey       1.00      1.00      1.00        69
               sci.crypt       1.00      0.99      0.99        70
         sci.electronics       0.99      1.00      0.99        66
                 sci.med       0.99      0.99      0.99        68
         

#3. Sentiment Analysis

In [33]:
df.head()

,Data,Labels
0,path frank odwyernewsgroups altatheismsubject ...,alt.atheism
1,newsgroups altatheismpath bilokcforumosrheedu ...,alt.atheism
2,path cantaloupesrvcscmuedudasnewsharvardedunoc...,alt.atheism
3,path jbrownbatmanbmdtrwcomnewsgroups altatheis...,alt.atheism
4,xref cantaloupesrvcscmuedu newsgroups altathei...,alt.atheism


In [34]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [35]:
!pip install afinn
from afinn import Afinn
# Assuming 'df' is your DataFrame
affinity_scores = df.set_index('Data')['Labels'].to_dict()
take(20, affinity_scores.items())

[('path frank odwyernewsgroups altatheismsubject islamic genocidedate apr gmtorganization siemensnixdorf aglines distribution worldmessageid references nntppostinghost article liveseysolntzewpdsgicom jon livesey writesin article frank odwyer writes article liveseysolntzewpdsgicom jon livesey writes noting particular society case mainland uk religously motivated murders murders kind says little whether interreligion murders elsewhere religiously motivated allows one conclude nothing inherent religion matter catholicism protestantism motivates one killmotivates allows christian bible says one may killunder certain circumstances fact instructs one kill undercertain circumstances id say majority people moral system instructs themto kill certain circumstances get distinction betweenmotivate allow agree flavour theism allowsatoricities thats indictment theism ratherdepends certain circumstances talk aboutchristianity islam least claims understoodits people go general statement theism fallsap

In [36]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # special operations on strings
import spacy # language models

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

import pandas
# Try specifying the delimiter, it might be a tab, semicolon, or some other character
book=pd.read_csv("/content/blogs.csv")  # Replace '\t' with the actual delimiter if known
#book = [x.strip() for x in book.x] # remove both the leading and the trailing characters - Original line causing the error
# The DataFrame has columns 'Data' and 'Labels'. Applying strip to the 'Data' column
book['Data'] = book['Data'].str.strip() # Apply strip to the 'Data' column
book = [x for x in book['Data'] if x] # removes empty strings, because they are considered in Python as False # Iterate through the 'Data' column

In [37]:
!pip install nltk
import nltk
nltk.download('punkt')

from nltk import tokenize
sentences = tokenize.sent_tokenize(" ".join(book))
sentences[5:15]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["I'd say the majority of people have a moral system that instructs them\nto kill under certain circumstances.",
 "I do get your distinction between\nmotivate and allow, and I do agree that if a flavour of theism 'allows'\natoricities, then that's an indictment of that theism.",
 "But it rather\ndepends on what the 'certain circumstances' are.",
 'When you talk about\nChristianity, or Islam, then at least your claims can be understood.',
 "It's when people go to a general statement about theism that it falls\napart.",
 'One could believe in a God which instructs one to be utterly\nharmless.',
 '#\n#|> For my part, I conclude that something\n#|> else is required.',
 'I also happen to believe that that something\n#|> else will work no less well without religion - any easy Them/Us will\n#|> do.',
 '#\n#And what does religion supply, if not an easy Them/Us?',
 'Not necessarily.']

In [38]:
sent_df = pd.DataFrame(sentences, columns=['sentence'])
sent_df

,sentence
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...
1,"#|> \n#|> No, but it allows one to conclude th..."
2,"#\n#""Motivates"" or ""allows?"""
3,The Christian Bible says that one may kill\n#u...
4,"In fact, it instructs one to kill under\n#cert..."
...,...
33919,Any evidence to the contrary ?
33920,"Bill R.\n\n--\n\n""When up a dangerous faction ..."
33921,"To ruin, slaughter, and confound; ..."
33922,"When such a tempest shook the land,\nHow could..."


In [39]:
#Custom function :score each word in a sentence in lemmatised form,
#but calculate the score for the whole original sentence.
!pip install spacy
import spacy
# Load the small English language model
nlp = spacy.load("en_core_web_sm")
sentiment_lexicon = affinity_scores

def calculate_sentiment(text: str = None) -> float:
    sent_score = 0
    if text:
        sentence = nlp(text)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score

In [40]:
calculate_sentiment(text = 'very sad')

0

In [41]:
sent_df['sentiment_value'] = sent_df['sentence'].apply(calculate_sentiment)

In [42]:
sent_df

,sentence,sentiment_value
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,0
1,"#|> \n#|> No, but it allows one to conclude th...",0
2,"#\n#""Motivates"" or ""allows?""",0
3,The Christian Bible says that one may kill\n#u...,0
4,"In fact, it instructs one to kill under\n#cert...",0
...,...,...
33919,Any evidence to the contrary ?,0
33920,"Bill R.\n\n--\n\n""When up a dangerous faction ...",0
33921,"To ruin, slaughter, and confound; ...",0
33922,"When such a tempest shook the land,\nHow could...",0


In [43]:
sent_df['word_count'] = sent_df['sentence'].str.split().apply(len)
sent_df['word_count'].head(10)

,word_count
0,94
1,32
2,4
3,11
4,10
5,18
6,28
7,10
8,15
9,14


In [44]:
sent_df.sort_values(by='sentiment_value').tail(10)

,sentence,sentiment_value,word_count
11305,">I'd try it on the VFR, but goddamn Competitio...",0,15
11304,">>My chain is lubed, my wheel is clean, after ...",0,10
11303,-- \n=> Dan DeClerck | ...,0,93
11302,This is a moot conversation.,0,5
11301,This difference of opinion is analogous to the...,0,15
11300,(sometimes it's hard for *any* motorcycle to f...,0,16
11299,"In other words,\nno one motorcycle can fufill ...",0,20
11298,"To ignore these, as well as other criteria, wo...",0,11
11312,If there's anything wet & stickey it will be c...,0,24
33923,- Jonathan Swift (1732),0,4


In [45]:
sent_df['sentiment_value'].describe()

,sentiment_value
count,33924.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [47]:
sent_df[sent_df['sentiment_value']<0].head()

,sentence,sentiment_value,word_count
